## Code to create simple sqlite dbase and adding data from organized datafiles

### Steps:

-

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Fri Dec 15 16:43:27 2017

@author: vpa
"""

import sqlite3
# Numerics
import numpy as np
import pandas as pd
import math

# import gvar as gv

### Create dbase and tables

In [3]:

dbase_fname="example.db"

conn=sqlite3.connect(dbase_fname)
c=conn.cursor()

# SQL codes to create tables


create_ensemble = """
    CREATE TABLE IF NOT EXISTS ensemble
    (
    id serial PRIMARY KEY,
    LX integer,
    LY integer,
    U numeric,
    log text,
    notes text,
    UNIQUE(LX, LY, U)
    );
    """
create_rhomono="""

    CREATE TABLE IF NOT EXISTS rho_mono
    (
    id serial PRIMARY KEY,
    ensid serial REFERENCES ensemble(id),
    rho float[],
    notes text,
    UNIQUE (id)
    );
    """
create_sus1="""
    CREATE TABLE IF NOT EXISTS sus1
    (
    id serial PRIMARY KEY,
    ensid serial REFERENCES ensemble(id),
    sus1 numeric,
    notes text,
    UNIQUE (id)
    );

    """        
    
create_sus2="""
    CREATE TABLE IF NOT EXISTS sus2
    (
    id serial PRIMARY KEY,
    ensid serial REFERENCES ensemble(id),
    sus2 numeric,
    notes text,
    UNIQUE (id)
    );

    """         
    
queries = [
    create_ensemble,
    create_rhomono,
    create_sus1,
    create_sus2
]


for query in queries:
    c.execute(query)

# Add data to dbase


##################
# Insert Data
##################


# Read line from Dbase







### Read data into lists

In [4]:

# Read data into list rws
num_files=3
rws=[[] for i in range(num_files)]
data_dir='../data/'
for idx,fname in enumerate(['RHO','SUS_1','SUS_2']):
    with open(data_dir+fname,'r') as f:
        rws[idx]=[i.rstrip('\n').split('\t') for i in f.readlines()]

# List storing entries to be written to 'ensemble' table
ens_lst=np.array([[idx,i[1],i[0]] for idx,i in enumerate(rws[0])])


###  Write data to Tables, row by row

In [5]:
conn=sqlite3.connect(dbase_fname)
c=conn.cursor()

n_vals=ens_lst.shape[0]

for i in range(n_vals):
    r1,r2,r3=rws[0][i],rws[1][i],rws[2][i]

    # Write to ensemble table
    a1=ens_lst[i]
    row_tup=tuple([a1[0],a1[1],a1[1],a1[2],'',''])
    insert_line="""INSERT INTO ensemble VALUES {}""".format(row_tup)
#     print insert_line
    c.execute(insert_line)    
    
    # Write into RHO,SUS_1,SUS_2 tables    
    l,u=r1[1],r1[0]
    L,U=int(float(l)),float(u)
    val1=[r1[2],r1[3]] # rho values
    val2=[r2[2],r2[3]] # sus1 values
    val3=[r3[2],r3[3]] # sus2 values


    for var,tname in zip([val1,val2,val3],['rho_mono','sus1','sus2']):
        
        # Find the  correct ensemble id from array ens_lst using u and l
        idx=ens_lst[(ens_lst[:,2]==u)&(ens_lst[:,1]==l)][0,0]

        row_tup=(idx,idx,",".join(var),'')
        insert_line="""INSERT INTO {0} VALUES {1}""".format(tname,row_tup)
#         print insert_line
        c.execute(insert_line)


conn.commit()
conn.close()

In [6]:
### Drop tables



### Read data from Database

In [7]:
# Read line from dbase


read_query=""" SELECT * FROM ensemble as es 
                JOIN rho_mono as rh ON (es.id=rh.ensid)
                JOIN sus1 ON (es.id=sus1.ensid)
                JOIN sus2 ON (es.id=sus2.ensid)
            """

# read_query=""" SELECT * FROM sus1           """    
conn=sqlite3.connect(dbase_fname)
c=conn.cursor()
conn.execute(read_query)

df=pd.read_sql_query(read_query,conn)

conn.commit()
conn.close()

df=df.T.drop_duplicates().T
df.head()


,id,LX,U,log,rho,sus1,sus2
0,0,16,0.43,,"0.0972468562312,0.000207017625181","2.08029779122,0.0026512173018","1.60081828214,0.00305809397142"
1,1,16,0.45,,"0.115162506256,0.000251014930975","2.18742711327,0.0025532798542","1.72478814471,0.00291429662038"
2,2,16,0.46,,"0.125995213964,0.000272384231147","2.23393131109,0.00254061995053","1.78067280409,0.00284546943731"
3,3,16,0.47,,"0.136936936937,0.000290944107501","2.26600614356,0.00236993107695","1.82325616301,0.00259678095915"
4,4,16,0.475,,"0.142689720971,0.0002925741102","2.27563236949,0.00231697794155","1.83811018774,0.00250221325235"


In [8]:
# ! rm example.db
# ! rm dbase_fname

    

### Problems:
- Need to write to dbase as list instead of string